Q2.  Consider a simple N-Queens problem.
  Take a suitable value of N and write a Python code for simulated annealing to solve the problem.  
  As we know, if we take T=0 in SA, it falls back to simple hill climbing search.  
  Modify the SA algorithm in order to make it an HC solution.  
  In the report, write proper theoretical justification in support of your modification (e.g., calculation of the acceptance probability) and the behavioral difference between the two approaches.  
  (Note: No marks will be given for simply implementing the HC solution.)

In [1]:
import random, math,copy
import pandas as pd
# import numpy as np
random.seed(42)
# np.random.seed(42)
# import copy

In [2]:
from IPython.display import display, HTML

def n_queens_to_html(board = [1, 3,0,2] ):
    """
    Generates an HTML table representation of an N-Queens board.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.

    Returns:
        str: HTML string for the N-Queens board.
    """
    n = len(board)
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .queen { background-color: #f4cccc; font-weight: bold; color: black; }
    </style>
    <table>
    """

    for i in range(n):
        html += "<tr>"
        for j in range(n):
            # Check if the current cell contains a queen
            if board[j] == i:
                html += '<td class="queen">Q</td>'
            else:
                html += "<td></td>"
        html += "</tr>"

    html += "</table>"
    return html

In [3]:
from IPython.display import display, HTML

def n_queens_to_html_with_conflicts(board, heuristic_dict):
    """
    Generates an HTML table representation of an N-Queens board with conflicts in blank cells.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        heuristic_dict (dict): Dictionary containing conflict information for each cell.

    Returns:
        str: HTML string for the N-Queens board.
    """
    n = len(board)
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .queen { background-color: #f4cccc; font-weight: bold; color: black; }
        .conflict { background-color: #f9f9f9; color: gray; }
    </style>
    <table>
    """

    for row in range(n):
        html += "<tr>"
        for col in range(n):
            conflicts = heuristic_dict[(row, col)]['conflicts']
            if heuristic_dict[(row, col)]['queen']:
                conflicts = conflicts if conflicts> 0 else ''
                # If a queen is present, display 'Q' with number of conflicts
                html += f'<td class="queen">Q {conflicts}</td>'
            else:
                # If no queen, display the number of conflicts
                html += f'<td class="conflict">{conflicts}</td>'
        html += "</tr>"

    html += "</table>"
    return html

# # Example usage
# if __name__ == "__main__":
#     n = 8  # Board size
#     board = [0, 4, 7, 5, 2, 6, 1, 3]  # Example board configuration


In [4]:
def get_new_board(n = 8):

    # Initialize the board randomly
    # The Queen's position on the board (row, column) =  (index,value) of the list
    # Each index represents a column, and the value at that index represents the row of the queen.

    board = [random.randint(0, n - 1) for i in range(n)]
    heuristic_dict = create_heuristic_dict(board, n)  # Generate the heuristic dictionary

    # Generate the HTML representation
    display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))
    return board, heuristic_dict

In [5]:
def create_heuristic_dict(board, n):
    """
    Creates a heuristic dictionary for the N-Queens board.
    The dictionary shows the total number of conflicts on the board
    if the queen from a specific column is moved to a particular position (row, col).

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).

    Returns:
        dict: A dictionary where each key is a tuple (row, col), and the value is a dictionary with:
              - 'queen': Whether a queen is currently present in the cell (True/False).
              - 'conflicts': Total number of conflicts on the board if the queen is moved to this cell.
    """
    heuristic_dict = {}

    for col in range(n):  # Iterate over each column
        original_row = board[col]  # Current row of the queen in this column
        for row in range(n):  # Iterate over each row in the column
            # Temporarily move the queen to the new position
            temp_board = board[:]
            temp_board[col] = row

            # Calculate the total number of conflicts for the new board
            conflicts = 0
            for queen_a in range(n):
                for queen_b in range(queen_a + 1, n):
                    # Check for conflicts (same row or same diagonal)
                    if temp_board[queen_a] == temp_board[queen_b] or \
                            abs(temp_board[queen_a] - temp_board[queen_b]) == abs(queen_a - queen_b):
                        conflicts += 1

            # Add the cell's data to the heuristic dictionary
            heuristic_dict[(row, col)] = {
                'queen': (original_row == row),  # True if the queen is currently in this cell
                'conflicts': conflicts
            }

    return heuristic_dict

In [ ]:
# Function to generate a random neighbor
def get_neighbor(board,n):

    neighbor_board = copy.deepcopy(board)

    # Randomly select a column to change
    col = random.randint(0, n - 1)
    
    while True:
        # Randomly select a new row for that column
        row = random.randint(0, n - 1)

        # Ensure the new row is different from the current row
        if row != board[col]:
    
            #set the new row value for the selected column
            neighbor_board[col] = row
    
            #return the new board
            return neighbor_board

In [ ]:
# Simulated Annealing algorithm
def simulated_annealing(n, initial_temperature, cooling_rate,board):


    # Calculate the initial number of conflicts
    current_conflicts = calculate_conflicts(board,n)
    temperature = initial_temperature

    while temperature > 0 and current_conflicts > 0:
        
        # Generate a neighbor board
        neighbor = get_neighbor(board,n)
        neighbor_conflicts = calculate_conflicts(neighbor,n)
        
        # Calculate the change in conflicts
        delta = neighbor_conflicts - current_conflicts

        # If neighbour has fewer conflicts accept neighbour
        # If the neighbour has more conflicts, then calculate probability of accepting worse solution
        # proabaility is given by e**(-delta / temperature)
        # A random number between 0 and 1 is generated, If this random number < probability, the worse solution is accepted.
        # When temperature is high, the probability of accepting worse solutions is higher.
        # As temperature decreases, the probability of accepting worse solutions decreases.
        
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            board = neighbor
            current_conflicts = neighbor_conflicts

        # Cool down
        temperature *= cooling_rate

    return board, current_conflicts

In [ ]:
display(HTML(n_queens_to_html(board)))

print("Simulated Annealing Solution:")
sa_solution, sa_conflicts = simulated_annealing(n=n, initial_temperature=100, cooling_rate=0.95,board=board)
print("Board:", sa_solution)
print("Conflicts:", sa_conflicts)
display(HTML(n_queens_to_html(sa_solution)))

In [ ]:
# Hill Climbing algorithm (modified SA with T=0)
def hill_climbing(n,board):
    current_conflicts = calculate_conflicts(board=board,n=n)

    while current_conflicts > 0:
        neighbor = get_neighbor(board=board,n=n)
        neighbor_conflicts = calculate_conflicts(board=neighbor,n=n)

        # Only accept better neighbors
        if neighbor_conflicts < current_conflicts:
            board = neighbor
            current_conflicts = neighbor_conflicts

    return board, current_conflicts

In [ ]:
heuristic_dict = {}
for row in range(n):
    for col in range(n):
        heuristic_dict[(row,col)] = 0

In [ ]:
display(HTML(n_queens_to_html(board)))

print("\nHill Climbing Solution:")
hc_solution, hc_conflicts = hill_climbing(n=n, board=board)
print("Board:", hc_solution)
print("Conflicts:", hc_conflicts)
display(HTML(n_queens_to_html(hc_solution)))

In [ ]:
def calculate_total_conflicts(board,n):
    """Helper function to calculate the total number of conflicts on the board."""
    heuristic_dict = create_heuristic_dict(board, n)
    total_conflicts = sum(
        heuristic_dict[(row, col)]['conflicts']
        for col in range(n)
        for row in range(n)
        if heuristic_dict[(row, col)]['queen']
    )
    return total_conflicts
def hill_climbing_with_heuristic(board, n):
    """
    Hill Climbing algorithm to solve the N-Queens problem using heuristic_dict.
    The algorithm evaluates moving each queen to a different row in its column
    and selects the move that reduces the total number of conflicts.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).

    Returns:
        tuple: A tuple containing the final board configuration and the number of conflicts.
    """


    current_conflicts = calculate_total_conflicts(board,n)

    while current_conflicts > 0:
        best_move = None
        best_conflicts = current_conflicts

        # Try moving each queen to a different row in its column
        for col in range(n):
            original_row = board[col]
            for row in range(n):
                if row == original_row:
                    continue  # Skip the current position of the queen

                # Create a new board with the queen moved
                new_board = board[:]
                new_board[col] = row

                # Calculate the total conflicts for the new board
                new_conflicts = calculate_total_conflicts(new_board,n)

                # If the new board has fewer conflicts, update the best move
                if new_conflicts < best_conflicts:
                    best_move = (col, row)
                    best_conflicts = new_conflicts

        # If no better move is found, terminate (local minimum)
        if best_move is None:
            break

        # Make the best move
        col, row = best_move
        board[col] = row
        current_conflicts = best_conflicts

        display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

    return board, current_conflicts

In [ ]:
n = 8  # Board size
board = [random.randint(0, n - 1) for _ in range(n)]  # Random initial board
print("Initial Board:", board)

# Solve using Hill Climbing
solution, conflicts = hill_climbing_with_heuristic(board, n)
print("Final Board:", solution)
print("Conflicts:", conflicts)

# Display the solution
heuristic_dict = create_heuristic_dict(solution, n)
display(HTML(n_queens_to_html_with_conflicts(solution, heuristic_dict)))

In [6]:
def calculate_total_conflicts(board,n):
    """Helper function to calculate the total number of conflicts on the board."""
    heuristic_dict = create_heuristic_dict(board, n)
    total_conflicts = sum(
        heuristic_dict[(row, col)]['conflicts']
        for col in range(n)
        for row in range(n)
        if heuristic_dict[(row, col)]['queen']
    )
    return total_conflicts
def hill_climbing_with_heuristic(board, n):
    """
    Hill Climbing algorithm to solve the N-Queens problem using heuristic_dict.
    The algorithm evaluates moving each queen to a different row in its column
    and selects the move that reduces the total number of conflicts.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).

    Returns:
        tuple: A tuple containing the final board configuration and the number of conflicts.
    """

    current_conflicts = calculate_total_conflicts(board,n)

    while current_conflicts > 0:
        best_move = None
        best_conflicts = current_conflicts

        # Try moving each queen to a different row in its column
        for col in range(n):
            original_row = board[col]
            for row in range(n):
                if row == original_row:
                    continue  # Skip the current position of the queen

                # Create a new board with the queen moved
                new_board = board[:]
                new_board[col] = row

                # Calculate the total conflicts for the new board
                new_conflicts = calculate_total_conflicts(new_board,n)

                # If the new board has fewer conflicts, update the best move
                if new_conflicts < best_conflicts:
                    best_move = (col, row)
                    best_conflicts = new_conflicts

        # If no better move is found, terminate (local minimum)
        if best_move is None:
            break

        # Make the best move
        col, row = best_move
        print(f"Moving column {col+1} Queen, from row {board[col]+1} to {row+1}")
        # col, row = best_move
        board[col] = row
        current_conflicts = best_conflicts

        # Display the updated board with conflicts
        heuristic_dict = create_heuristic_dict(board, n)
        display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

    return board, current_conflicts

In [7]:
n=8
board, heuristic_dict = get_new_board(n = n)
solution, conflicts = hill_climbing_with_heuristic(board, n)
print("Final Board:", solution)
print("Conflicts:", conflicts)
# print(f"Moving Queen {col} from row {original_row} to {row} reduces conflicts from {current_conflicts} to {best_conflicts}")

9,Q 10,10,10,7,10,10,12
Q 10,12,14,11,10,13,Q 10,Q 10
9,10,11,11,9,Q 10,8,10
9,11,12,Q 10,Q 10,11,9,10
8,9,Q 10,13,9,11,7,10
7,10,12,10,7,9,8,8
9,11,10,8,7,9,7,10
10,9,9,8,6,9,7,9


Moving column 5 Queen, from row 4 to 8


5,Q 6,6,7,7,7,7,7
Q 6,9,9,8,10,10,Q 6,Q 6
5,7,7,7,9,Q 6,5,6
5,7,7,Q 6,10,7,5,5
4,7,Q 6,9,9,7,4,7
3,7,8,7,7,6,5,4
5,7,6,6,7,7,4,5
6,7,6,6,Q 6,7,5,6


Moving column 1 Queen, from row 2 to 6


5,Q 3,3,4,4,5,5,5
6,6,5,4,7,6,Q 3,Q 3
5,4,4,5,6,Q 3,3,4
5,5,4,Q 3,7,4,3,3
4,6,Q 3,5,6,4,2,5
Q 3,6,6,5,4,4,4,3
5,6,3,3,4,3,2,3
6,5,4,3,Q 3,4,2,4


Moving column 7 Queen, from row 2 to 5


4,Q 2,2,3,3,4,5,4
4,4,2,3,5,5,3,Q 2
4,3,2,4,6,Q 2,3,3
4,4,2,Q 2,5,5,3,4
4,6,Q 2,4,6,5,Q 2,6
Q 2,5,3,4,3,5,4,4
4,4,1,2,4,3,2,3
4,4,2,3,Q 2,4,2,4


Moving column 3 Queen, from row 5 to 7


3,Q 1,2,3,2,3,4,2
3,3,2,3,4,3,3,Q 1
2,2,2,4,4,Q 1,4,1
3,2,2,Q 1,4,5,3,2
3,4,2,3,5,3,Q 1,3
Q 1,4,3,4,2,4,4,2
3,4,Q 1,3,3,3,3,2
3,4,2,4,Q 1,2,2,2


Final Board: [5, 0, 6, 3, 7, 2, 4, 1]
Conflicts: 8


Explanation:
Dynamic Conflict Calculation:

For each column, iterate over all possible rows where the queen could be moved.
Temporarily move the queen to the new position and calculate the total number of conflicts for the entire board.
Heuristic Dictionary:

Each key is a tuple (row, col) representing a cell.
The value is a dictionary with:
'queen': Whether the queen is currently in this cell.
'conflicts': The total number of conflicts on the board if the queen is moved to this cell.
Conflict Calculation:

For each pair of queens (queen_a, queen_b) on the board, check if they are in conflict:
Same row: temp_board[queen_a] == temp_board[queen_b]
Same diagonal: abs(temp_board[queen_a] - temp_board[queen_b]) == abs(queen_a - queen_b)
Temporary Board:

A temporary board (temp_board) is used to simulate moving the queen to a new position without modifying the original board.

Updated hill_climbing_with_heuristic Function:
The hill_climbing_with_heuristic function will use the updated create_heuristic_dict to evaluate moves based on the total number of conflicts.

In [ ]:
def simulated_annealing_with_heuristic(board, n, initial_temperature, cooling_rate):
    """
    Simulated Annealing algorithm to solve the N-Queens problem using heuristic_dict.
    The algorithm evaluates moving each queen to a position one step away in its column
    and probabilistically accepts moves based on the temperature.

    Args:
        board (list of int): 1D list representing the N-Queens board.
                             Each index represents a column, and the value at that index represents the row of the queen.
        n (int): Size of the board (N x N).
        initial_temperature (float): Initial temperature for the simulated annealing algorithm.
        cooling_rate (float): Cooling rate for the temperature.

    Returns:
        tuple: A tuple containing the final board configuration and the number of conflicts.
    """
    current_conflicts = calculate_total_conflicts(board, n)
    temperature = initial_temperature

    while temperature > 0 and current_conflicts > 0:

    current_conflicts = calculate_total_conflicts(board,n)

    while current_conflicts > 0:
        best_move = None
        best_conflicts = current_conflicts

        # Try moving each queen to a different row in its column
        for col in range(n):
            original_row = board[col]
            for row in range(n):
                if row == original_row:
                    continue  # Skip the current position of the queen

                # Create a new board with the queen moved
                new_board = board[:]
                new_board[col] = row

                # Calculate the total conflicts for the new board
                new_conflicts = calculate_total_conflicts(new_board,n)

                # If the new board has fewer conflicts, update the best move
                if new_conflicts < best_conflicts:
                    best_move = (col, row)
                    best_conflicts = new_conflicts

        # If no better move is found, terminate (local minimum)
        if best_move is None:
            break

        # Make the best move
        col, row = best_move
        print(f"Moving column {col+1} Queen, from row {board[col]+1} to {row+1}")
        # col, row = best_move
        board[col] = row
        current_conflicts = best_conflicts

        # Display the updated board with conflicts
        heuristic_dict = create_heuristic_dict(board, n)
        display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

    return board, current_conflicts

In [9]:
solution, conflicts = simulated_annealing_with_heuristic(board, n, initial_temperature=100, cooling_rate=0.95)
print("Final Board:", solution)
print("Conflicts:", conflicts)

Final Board: [5, 0, 6, 3, 7, 2, 4, 1]
Conflicts: 8


In [ ]:
# def simulated_annealing_with_heuristic(board, n, initial_temperature, cooling_rate):
#     """
#     Simulated Annealing algorithm to solve the N-Queens problem using heuristic_dict.
#     The algorithm evaluates moving each queen to a position one step away in its column
#     and probabilistically accepts moves based on the temperature.

#     Args:
#         board (list of int): 1D list representing the N-Queens board.
#                              Each index represents a column, and the value at that index represents the row of the queen.
#         n (int): Size of the board (N x N).
#         initial_temperature (float): Initial temperature for the simulated annealing algorithm.
#         cooling_rate (float): Cooling rate for the temperature.

#     Returns:
#         tuple: A tuple containing the final board configuration and the number of conflicts.
#     """
#     current_conflicts = calculate_total_conflicts(board, n)
#     temperature = initial_temperature

#     while temperature > 0 and current_conflicts > 0:
#         # Randomly select a column
#         col = random.randint(0, n - 1)
#         current_row = board[col]

#         # Generate potential moves (one step away in the column)
#         potential_moves = [current_row - 1, current_row + 1]
#         potential_moves = [row for row in potential_moves if 0 <= row < n]

#         if not potential_moves:
#             continue

#         # Randomly select a new row from potential moves
#         new_row = random.choice(potential_moves)

#         # Create a new board with the queen moved
#         new_board = board[:]
#         new_board[col] = new_row

#         # Calculate the total conflicts for the new board
#         new_conflicts = calculate_total_conflicts(new_board, n)

#         # Calculate the change in conflicts
#         delta = new_conflicts - current_conflicts

#         # Decide whether to accept the move
#         if delta < 0 or random.random() < math.exp(-delta / temperature):
#             board = new_board
#             current_conflicts = new_conflicts

#         # Display the updated board with conflicts
#         print(f"Moving column {col+1} Queen, from row {board[col]+1} to {new_row+1}")
#         heuristic_dict = create_heuristic_dict(board, n)
#         display(HTML(n_queens_to_html_with_conflicts(board, heuristic_dict)))

#         # Cool down the temperature
#         temperature *= cooling_rate

#     return board, current_conflicts

In [ ]:
solution, conflicts = simulated_annealing_with_heuristic(board, n, initial_temperature=100, cooling_rate=0.95)
print("Final Board:", solution)
print("Conflicts:", conflicts)